In [1]:
import sys
import os

# Set the main path in the root folder of the project.
sys.path.append(os.path.join('..'))

In [2]:
# Settings for autoreloading.
%load_ext autoreload
%autoreload 2

In [3]:
from src.utils.seed import set_random_seed

# Set the random seed for deterministic operations.
SEED = 42
set_random_seed(SEED)

In [4]:
import torch

# Set the device for training and querying the model.
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'The selected device is: "{DEVICE}"')

The selected device is: "cuda"


# Loading the data

In [8]:
import os

RAW_DATA_DIR = os.path.join('..', 'data', 'pems-bay', 'raw')
PROCESSED_DATA_DIR = os.path.join('..', 'data', 'pems-bay', 'processed')
PREDICTED_DATA_DIR = os.path.join('..', 'data', 'pems-bay', 'predicted')

In [6]:
import pickle

# Get the data scaler.
with open(os.path.join(PROCESSED_DATA_DIR, 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)

In [9]:
from src.spatial_temporal_gnn.model import SpatialTemporalGNN
from src.data.data_extraction import get_adjacency_matrix

# Get the adjacency matrix
adj_matrix_structure = get_adjacency_matrix(
    os.path.join(RAW_DATA_DIR, 'adj_mx_pems_bay.pkl'))

# Get the header of the adjacency matrix and the matrix itself.
header, _, adj_matrix = adj_matrix_structure

# Get the STGNN and load the checkpoints.
spatial_temporal_gnn = SpatialTemporalGNN(9, 1, 12, 12, adj_matrix, DEVICE, 64)

stgnn_checkpoints_path = os.path.join('..', 'models', 'checkpoints',
                                      'st_gnn_pems_bay.pth')

stgnn_checkpoints = torch.load(stgnn_checkpoints_path)
spatial_temporal_gnn.load_state_dict(stgnn_checkpoints['model_state_dict'])

# Set the model in evaluation mode.
spatial_temporal_gnn.eval();

In [10]:
import os
import numpy as np
from src.spatial_temporal_gnn.prediction import predict

# Get the data and the values predicted by the STGNN.
x_train = np.load(os.path.join(PREDICTED_DATA_DIR, 'x_train.npy'))
y_train = np.load(os.path.join(PREDICTED_DATA_DIR, 'y_train.npy'))
x_val = np.load(os.path.join(PREDICTED_DATA_DIR, 'x_val.npy'))
y_val = np.load(os.path.join(PREDICTED_DATA_DIR, 'y_val.npy'))
x_test = np.load(os.path.join(PREDICTED_DATA_DIR, 'x_test.npy'))
y_test = np.load(os.path.join(PREDICTED_DATA_DIR, 'y_test.npy'))

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\pems-bay\\predicted\\x_train.npy'

# Map the event set to the graph

In [ ]:
from src.explanation.navigator.model import Navigator

model = Navigator(DEVICE)

c:\Users\riccardo\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
from src.explanation.navigator.dataloaders import get_dataloader

train_loader = get_dataloader(x_train, y_train, batch_size=None, shuffle=True)
val_loader = get_dataloader(x_val, y_val, batch_size=None, shuffle=False)
test_loader = get_dataloader(x_test, y_test, batch_size=None, shuffle=False)

In [ ]:
#x  = next(iter(train_loader))


In [ ]:
from src.spatial_temporal_gnn.training import Checkpoint

optimizer = torch.optim.Adam(model.parameters(), lr=1e-6, weight_decay=2e-8)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.94, verbose=False)

checkpoint_file_path = os.path.join('..', 'models', 'checkpoints',
                                    'navigator_pems_bay.pth')
checkpoint = Checkpoint(checkpoint_file_path)

EPOCHS = 5
VALIDATIONS_PER_EPOCH = 4

In [ ]:
from src.explanation.navigator.training import train

history = train(
    model, optimizer, train_loader, val_loader, spatial_temporal_gnn, scaler,
    EPOCHS, VALIDATIONS_PER_EPOCH, checkpoint, lr_scheduler,
    reload_best_weights=True)

Epoch 1/5
[247/988] - 189s 763ms/step - train { MAE (loss): 2.99 - RMSE: 2.99 - MAPE: 7.62% } - lr: 1e-06 - weight decay: 2e-08              
	val step - val: { MAE: 2.97 - RMSE: 2.97 - MAPE: 7.45% } - lr: 1e-06 - weight decay: 2e-08
[494/988] - 432s 874ms/step - train { MAE (loss): 3.13 - RMSE: 3.13 - MAPE: 8.16% } - lr: 1e-06 - weight decay: 2e-08              
	val step - val: { MAE: 2.73 - RMSE: 2.73 - MAPE: 6.4% } - lr: 1e-06 - weight decay: 2e-08
[741/988] - 672s 907ms/step - train { MAE (loss): 3.19 - RMSE: 3.19 - MAPE: 8.15% } - lr: 1e-06 - weight decay: 2e-08              
	val step - val: { MAE: 2.76 - RMSE: 2.76 - MAPE: 5.62% } - lr: 1e-06 - weight decay: 2e-08
[988/988] - 909s 920ms/step - train { MAE (loss): 3.2 - RMSE: 3.2 - MAPE: 8.1% } - lr: 1e-06 - weight decay: 2e-08                 
	val step - val: { MAE: 2.63 - RMSE: 2.63 - MAPE: 7.29% } - lr: 1e-06 - weight decay: 2e-08
[988/988] - 909s - train: { MAE (loss): 3.2 - RMSE: 3.2 - MAPE: 8.1% } - val: { MAE: 2.94 - RMS

In [ ]:
from src.explanation.navigator.training import validate

test_mae, test_rmse, test_mape = validate(
    model, test_loader, spatial_temporal_gnn, scaler)

print(f'Results on the test set: MAE {test_mae:.3g} - RMSE {test_rmse:.3g}',
      f'- MAPE {test_mape * 100.:.3g} %')

Results on the test set: MAE 2.63 - RMSE 2.63 - MAPE 6.44 %


In [ ]:
x, ev, t, y = next(iter(train_loader))

In [ ]:
t_repeated = t.unsqueeze(0).repeat(ev.shape[0], 1).to(device=DEVICE).float()
res = model(ev.to(DEVICE).float(), t_repeated)


ev_scores = torch.zeros((x.shape[0], x.shape[1], 1))
            
for i, x_ in enumerate(ev):
    timestep = int(x_[0].item()); node = int(x_[1].item())
    ev_scores[timestep, node, 0] = res[i]

In [ ]:
len(torch.unique(ev_scores.sigmoid()))

2463

In [ ]:
for ev in torch.unique(ev_scores.sigmoid()):
    print(ev)

tensor(0.0830, grad_fn=<UnbindBackward0>)
tensor(0.0849, grad_fn=<UnbindBackward0>)
tensor(0.0928, grad_fn=<UnbindBackward0>)
tensor(0.0968, grad_fn=<UnbindBackward0>)
tensor(0.1026, grad_fn=<UnbindBackward0>)
tensor(0.1058, grad_fn=<UnbindBackward0>)
tensor(0.1089, grad_fn=<UnbindBackward0>)
tensor(0.1106, grad_fn=<UnbindBackward0>)
tensor(0.1115, grad_fn=<UnbindBackward0>)
tensor(0.1142, grad_fn=<UnbindBackward0>)
tensor(0.1161, grad_fn=<UnbindBackward0>)
tensor(0.1169, grad_fn=<UnbindBackward0>)
tensor(0.1272, grad_fn=<UnbindBackward0>)
tensor(0.1279, grad_fn=<UnbindBackward0>)
tensor(0.1347, grad_fn=<UnbindBackward0>)
tensor(0.1350, grad_fn=<UnbindBackward0>)
tensor(0.1354, grad_fn=<UnbindBackward0>)
tensor(0.1379, grad_fn=<UnbindBackward0>)
tensor(0.1387, grad_fn=<UnbindBackward0>)
tensor(0.1429, grad_fn=<UnbindBackward0>)
tensor(0.1448, grad_fn=<UnbindBackward0>)
tensor(0.1451, grad_fn=<UnbindBackward0>)
tensor(0.1470, grad_fn=<UnbindBackward0>)
tensor(0.1476, grad_fn=<UnbindBack